# Importación de librerías.

## Requests realiza la petición al servidor.

## BeautifulSoup analizar documentos HTML.

## Pandas podemos representar datos tabulares con columnas con etiquetas y filas y series temporales.

In [43]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Se realiza un request de la url_inicial de la revista, url_root nos ayuda a genera un url completa a partir de la url raíz para esto se utiliza la librería urljoin.

La revista que se está scrapendo es la siguiente http://www.revistarelaciones.com/index.php/relaciones/index

In [44]:
url_inicial='http://www.revistarelaciones.com/index.php/relaciones/issue/archive'
url_root= 'http://www.revistarelaciones.com/index.php/relaciones/issue/archive'
r = requests.get(url_inicial)

In [45]:
soup = BeautifulSoup(r.text, 'html.parser')

Se realiza la búsqueda para obtener los urls de cada uno de los volúmenes o archivos de la revista.

In [46]:
box = soup.find('div', id='issues')
volumen=box.findAll('h4')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://www.revistarelaciones.com/index.php/relaciones/issue/view/80',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/73',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/74',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/72',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/71',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/79',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/78',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/77',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/76',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/75',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/47',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/46',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/45',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/vi

Se realiza una lista en la que la variable vol que contiene las urls de cada uno de los volúmenes publicados de la revista, con estos urls obtenidos,  con la variable vol se implementa un ciclo en el cual se utiliza la variable vol2 de la cual se van acumulando las siguientes urls.

In [48]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.findAll('div', id='issueCoverImage') 
    vol1 = [x.find('a').get('href')for x in box1]
    vol2+=vol1

In [49]:
vol2

['http://www.revistarelaciones.com/index.php/relaciones/issue/view/80/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/73/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/74/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/72/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/71/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/79/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/77/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/76/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/75/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/47/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/46/showToc',
 'http://www.revistarelaciones.com/index.php/relaciones/issue/view/45/showToc',
 'http://www.revistarelaciones.com/index

Algunas revistas necesitan más búsquedas para llega a los artículos ya que algunos manejan primero los volúmenes o archivos y de eso archivos se dirigen a un urls que es la tabla contenidos, donde se encuentran los artículos, por lo cual se comienza a realizar la misma búsqueda que en vol2.

Se implementa otro ciclo con vol2, para esto se utiliza la variable de vol3 que recolecta las urls de los artículos, para comenzar con su scrapeo.

In [50]:
vol3 =[]
for b in vol2: 
    url_inicial2=b 
    r2 = requests.get(url_inicial2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('tr', valign='top')
    vol2= [o.find('a').get('href')for o in volumen2]
    vol3+=vol2

In [51]:
vol3

['http://www.revistarelaciones.com/index.php/relaciones/article/view/861',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/845',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/856',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/843',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/807',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/873',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/766',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/719',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/788',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/702',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/676',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/767',
 'http://www.revistarelaciones.com/index.php/relaciones/article/view/768',
 'http://www.revistarelac

En esta función variable sopa se utiliza para tener las paginaciones de la revista y la variable url es guardan cada uno de url que ya se obtuvieron de vol3.

In [52]:
def get_url_items(sopa, url): 
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('tr', valign='top')
    vol2= [o.find('a').get('href')for o in volumen2]
    return vol3

Ahora se va acumulando cada uno de los links e ir iterando en cada una de las páginas, para traer cada uno de los links que se van a ir scrapeando.

In [53]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina http://www.revistarelaciones.com/index.php/relaciones/issue/archive


Obtiene el número de los artículos que se encontraron en los primeros volúmenes de la revista.

In [54]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

588

Toma uno a uno de los links donde se encuentra la información de cada artículo para Scrapearlos.

In [55]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

Mediante esta función revisa cada uno de los links de los artículos en los cuales se encuentra la información
del artículo, por lo cual se aplican cada uno de los métodos, que nos van a obtener lo que estamos requiriendo de dicho artículo.

In [56]:
def scraper_book(url):
    content_book={}# diccionario que no va acumulando la imformacion que vamos poteniddo para el tada frame
    r=requests.get(url)
    tr='Relaciones Estudios de Historia y Sociedad'
    a='IV. Humanidades y Ciencias de la Conducta'
    te='Multidisciplinaria'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h3').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    #obtencion de Resumen
    try:
        des=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=des.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #obtencion de Abstract
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    #obtener link Articulos con la url, donde se guarda los urls donde se encutran la informacion de los articulos.
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
    #obtener link PDF
    try:
        ancla_link=s_item.find('div', id='articleFullText').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

El list_scraper hace un scrapeo de la cada uno de los artículos, ya que en datos_book se acumuló la información obtenida de lo métodos anteriores.

In [57]:
list_scraper=list_scraper[0:10]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9


La variable datos_book es un listado en el que se recolecto la información requerida por lo cual
se convierte en un DataFrame.

In [58]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Orfandad y violencia a niñas, niños y adolesce...",La pandemia de Covid-19 ha desencadenado en Me...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
1,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Identidades de género en álbumes fotográficos ...,El presente texto analiza las características ...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
2,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Influencia de factores migratorios en la inten...,Objetivo: analizar por medio de la Teoría del ...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
3,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,La epidemia de viruela de 1782-1783 y sus efec...,La epidemia que asoló a Santafé -Bogotá- y sus...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
4,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Etnografía de la nostalgía. La crisis del opio...,El artículo analiza los efectos de la caída de...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
5,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Socialización de información y miedo al crimen...,El miedo al crimen es un problema que nace y s...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
6,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Urbanización informal y ciudad desigual, contr...",El objetivo de este trabajo es analizar la din...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
7,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Territorio confiscado para la conservación: lo...,Este artículo revisa la producción social de u...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
8,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Entre lo legal, lo ilícito y lo clandestino. P...",Reseña de libro,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
9,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,El gobernante en Mesoamérica. Representaciones...,El libro presenta cinco capitulos que conforma...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...


En el caso de esta revista solo de deseaba obtener los artículos, por lo cual
en algunas ocasiones se obtienes cosas diferentes, para esto con la función isin lo que hace es 
eliminar mediante el Titulo Articulo los archivos que no se desean.

In [59]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","Enlaces refback","Mañana o pasado","Pura imagen","Número completo", None])]
df

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Orfandad y violencia a niñas, niños y adolesce...",La pandemia de Covid-19 ha desencadenado en Me...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
1,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Identidades de género en álbumes fotográficos ...,El presente texto analiza las características ...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
2,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Influencia de factores migratorios en la inten...,Objetivo: analizar por medio de la Teoría del ...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
3,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,La epidemia de viruela de 1782-1783 y sus efec...,La epidemia que asoló a Santafé -Bogotá- y sus...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
4,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Etnografía de la nostalgía. La crisis del opio...,El artículo analiza los efectos de la caída de...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
5,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Socialización de información y miedo al crimen...,El miedo al crimen es un problema que nace y s...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
6,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Urbanización informal y ciudad desigual, contr...",El objetivo de este trabajo es analizar la din...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
7,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,Territorio confiscado para la conservación: lo...,Este artículo revisa la producción social de u...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
8,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,"Entre lo legal, lo ilícito y lo clandestino. P...",Reseña de libro,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...
9,Relaciones Estudios de Historia y Sociedad,IV. Humanidades y Ciencias de la Conducta,Multidisciplinaria,El gobernante en Mesoamérica. Representaciones...,El libro presenta cinco capitulos que conforma...,None,http://www.revistarelaciones.com/index.php/rel...,http://www.revistarelaciones.com/index.php/rel...


La información solicitada se pasa a un CSV.

In [60]:
df.to_csv('Revista154.csv', index=False)

Final del Código